### trade based simulation

# data import using yfinance

## 라이브러리 임포트

In [1]:
!pip install yfinance
!pip install pandas-datareader

     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 6.4 MB 20.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import yfinance as yf
import pandas_datareader as pdr

In [3]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

## 데이터 임포트

In [4]:
tickers = ["SPY", "QQQ", "GLD", "LQD", "EEM"]
risk_free_asset = "TB3MS"

In [5]:
divs = []
for ticker in tickers:
  div = yf.Ticker(ticker).dividends
  div.index = pd.to_datetime(div.index).strftime('%Y-%m')
  if not div.index.is_unique:
    div = div.groupby(div.index).sum()
  divs.append(div)

In [6]:
dividends_df = pd.concat(divs, axis=1)
dividends_df.columns = [["Dividend"] * len(tickers), tickers]
dividends_df.index.name = "Date"
dividends_df.sort_index(inplace=True)

In [7]:
dividends_df

Dividend                         
             SPY    QQQ GLD    LQD    EEM
Date                                     
1993-03    0.213    NaN NaN    NaN    NaN
1993-06    0.318    NaN NaN    NaN    NaN
1993-09    0.286    NaN NaN    NaN    NaN
1993-12    0.317    NaN NaN    NaN    NaN
1994-03    0.271    NaN NaN    NaN    NaN
...          ...    ...  ..    ...    ...
2021-09    1.428  0.414 NaN  0.234    NaN
2021-10      NaN    NaN NaN  0.255    NaN
2021-11      NaN    NaN NaN  0.262    NaN
2021-12    1.633  0.491 NaN  0.495  0.733
2022-02      NaN    NaN NaN  0.242    NaN

[251 rows x 5 columns]

In [8]:
tickers_df = yf.download(" ".join(tickers), period='max', interval='1mo')

[*********************100%***********************]  5 of 5 completed


In [9]:
tickers_df.index = pd.to_datetime(tickers_df.index).strftime('%Y-%m')

In [10]:
tickers_df.drop(columns=["Adj Close", "High", "Low", "Open", "Volume"], inplace=True)
tickers_df.dropna(how="all", inplace=True)

In [11]:
if not tickers_df.index.is_unique:
    tickers_df = tickers_df.groupby(tickers_df.index).last()

In [12]:
# tickers_df = pd.concat([tickers_df, dividends_df], axis=1)

In [13]:
# tickers_df.columns = tickers_df.columns.swaplevel(0,1)
# tickers_df.sort_index(axis=1, level=0, inplace=True)

In [14]:
tb3ms_df = pdr.DataReader(risk_free_asset, "fred", tickers_df.index[0], tickers_df.index[-1] )
tb3ms_df["RoR12"] = tb3ms_df[risk_free_asset].map(lambda x: x / 100)
tb3ms_df["CRoR"] = (1 + tb3ms_df["RoR12"] / 12).cumprod()
tb3ms_df.index = pd.to_datetime(tb3ms_df.index).strftime('%Y-%m')

In [15]:
tb3ms_df

,TB3MS,RoR12,CRoR
DATE,,,
1993-02,2.93,0.0293,1.002442
1993-03,2.95,0.0295,1.004906
1993-04,2.87,0.0287,1.007309
1993-05,2.96,0.0296,1.009794
1993-06,3.07,0.0307,1.012377
...,...,...,...
2021-09,0.04,0.0004,1.912639
2021-10,0.05,0.0005,1.912718
2021-11,0.05,0.0005,1.912798


In [16]:
tickers_df[("Close", risk_free_asset)] = tb3ms_df["CRoR"]

In [17]:
tickers_df

Close                                                          
               EEM         GLD         LQD         QQQ         SPY     TB3MS
Date                                                                        
1993-02        NaN         NaN         NaN         NaN   44.406250  1.002442
1993-03        NaN         NaN         NaN         NaN   45.187500  1.004906
1993-04        NaN         NaN         NaN         NaN   44.031250  1.007309
1993-05        NaN         NaN         NaN         NaN   45.218750  1.009794
1993-06        NaN         NaN         NaN         NaN   45.062500  1.012377
...            ...         ...         ...         ...         ...       ...
2021-10  50.919998  166.649994  133.479996  386.109985  459.250000  1.912718
2021-11  48.840000  165.500000  133.059998  393.820007  455.559998  1.912798
2021-12  48.849998  170.960007  132.520004  397.850006  474.959991  1.912894
2022-01  48.840000  168.089996  127.760002  363.049988  449.910004  1.913133
2022-02  48.720001  173.809998  124.839996  347.059998  440.459991       NaN

[349 rows x 6 columns]

# backtesting

In [18]:
exclude_period, look_back = 3, 12

In [19]:
ror_df = tickers_df.apply(lambda x: round((x.shift(exclude_period) - x.shift(look_back)) / x.shift(look_back), 4), axis=0)

In [20]:
ror_df.columns = ror_df.columns.map(lambda x: x[1])

In [21]:
# tickers_df = pd.concat([tickers_df, ror_df], axis=1)

In [22]:
ror_df

,EEM,GLD,LQD,QQQ,SPY,TB3MS
Date,,,,,,
1993-02,NaN,NaN,NaN,NaN,NaN,NaN
1993-03,NaN,NaN,NaN,NaN,NaN,NaN
1993-04,NaN,NaN,NaN,NaN,NaN,NaN
1993-05,NaN,NaN,NaN,NaN,NaN,NaN
1993-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-10,0.1541,-0.0362,0.0170,0.3534,0.3429,0.0004
2021-11,0.0755,0.0181,-0.0230,0.2681,0.2472,0.0004
2021-12,-0.0250,-0.0793,-0.0369,0.1409,0.1478,0.0003


In [23]:
# ror_df.max(axis=1)

In [24]:
winners = ror_df.idxmax(axis=1)

In [25]:
winners[:look_back] = risk_free_asset

In [26]:
# idx, cols = pd.factorize(winners)
# cols = cols.map(lambda x: ("Close", x))
# tickers_df.reindex(cols, axis=1).to_numpy()[np.arange(len(idx)), idx]

In [27]:
winners = winners.to_numpy()

In [28]:
winners

array(['TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS',
       'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'SPY', 'SPY', 'SPY',
       'SPY', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'SPY', 'TB3MS',
       'TB3MS', 'TB3MS', 'TB3MS', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'SPY', 'TB3MS', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY', 'SPY',
       'SPY', 'QQQ', 'QQQ', 'QQQ', 'QQQ', 'QQQ', 'QQQ', 'QQQ', 'QQQ',
       'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS',
       'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS',
       'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS', 'TB3MS',
       'TB3